In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import test
from util import *

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 25: Full of Hot Air ---</h2><p>As the expedition finally reaches the extraction point, several large <a href="https://en.wikipedia.org/wiki/Hot_air_balloon" target="_blank">hot air balloons</a> drift down to meet you. Crews quickly start unloading the equipment the balloons brought: many hot air balloon kits, some fuel tanks, and a <em>fuel heating machine</em>.</p>
<p>The fuel heating machine is a new addition to the process. When this mountain was a volcano, the ambient temperature was more reasonable; now, it's so cold that the fuel won't work at all without being warmed up first.</p>
<p>The Elves, seemingly in an attempt to make the new machine feel welcome, have already attached a pair of <a href="https://en.wikipedia.org/wiki/Googly_eyes" target="_blank">googly eyes</a> and started calling it "Bob".</p>
<p>To heat the fuel, Bob needs to know the total amount of fuel that will be processed ahead of time so it can correctly calibrate heat output and flow rate. This amount is simply the <em>sum</em> of the fuel requirements of all of the hot air balloons, and those fuel requirements are even listed clearly on the side of each hot air balloon's burner.</p>
<p>You assume the Elves will have no trouble adding up some numbers and are about to go back to figuring out which balloon is yours when you get a tap on the shoulder. Apparently, the fuel requirements use numbers written in a format the Elves don't recognize; predictably, they'd like your help deciphering them.</p>
<p>You make a list of all of the fuel requirements (your puzzle input), but you don't recognize the number format either. For example:</p>
<pre><code>1=-0-2
12111
2=0=
21
2=01
111
20012
112
1=-1=
1-12
12
1=
122
</code></pre>
<p>Fortunately, Bob is labeled with a support phone number. Not to be deterred, you call and ask for help.</p>
<p>"That's right, just supply the fuel amount to the-- oh, for more than one burner? No problem, you just need to add together our Special Numeral-Analogue Fuel Units. Patent pending! They're way better than normal numbers for--"</p>
<p>You mention that it's quite cold up here and ask if they can skip ahead.</p>
<p>"Okay, our Special Numeral-Analogue Fuel Units - SNAFU for short - are sort of like normal numbers. You know how starting on the right, normal numbers have a ones place, a tens place, a hundreds place, and so on, where the digit in each place tells you how many of that value you have?"</p>
<p>"SNAFU works the same way, except it uses powers of five instead of ten. Starting from the right, you have a ones place, a fives place, a twenty-fives place, a one-hundred-and-twenty-fives place, and so on. It's that easy!"</p>
<p>You ask why some of the digits look like <code>-</code> or <code>=</code> instead of "digits".</p>
<p>"You know, I never did ask the engineers why they did that. Instead of using digits four through zero, the digits are <code><em>2</em></code>, <code><em>1</em></code>, <code><em>0</em></code>, <em>minus</em> (written <code>-</code>), and <em>double-minus</em> (written <code>=</code>). Minus is worth -1, and double-minus is worth -2."</p>
<p>"So, because ten (in normal numbers) is two fives and no ones, in SNAFU it is written <code>20</code>. Since eight (in normal numbers) is two fives minus two ones, it is written <code>2=</code>."</p>
<p>"You can do it the other direction, too. Say you have the SNAFU number <code>2=-01</code>. That's <code>2</code> in the 625s place, <code>=</code> (double-minus) in the 125s place, <code>-</code> (minus) in the 25s place, <code>0</code> in the 5s place, and <code>1</code> in the 1s place. (2 times 625) plus (-2 times 125) plus (-1 times 25) plus (0 times 5) plus (1 times 1). That's 1250 plus -250 plus -25 plus 0 plus 1. <em>976</em>!"</p>
<p>"I see here that you're connected via our premium uplink service, so I'll transmit our handy SNAFU brochure to you now. Did you need anything else?"</p>
<p>You ask if the fuel will even work in these temperatures.</p>
<p>"Wait, it's <em>how</em> cold? There's no <em>way</em> the fuel - or <em>any</em> fuel - would work in those conditions! There are only a few places in the-- where did you say you are again?"</p>
<p>Just then, you notice one of the Elves pour a few drops from a snowflake-shaped container into one of the fuel tanks, thank the support representative for their time, and disconnect the call.</p>
<p>The SNAFU brochure contains a few more examples of decimal ("normal") numbers and their SNAFU counterparts:</p>
<pre><code>  Decimal          SNAFU
        1              1
        2              2
        3             1=
        4             1-
        5             10
        6             11
        7             12
        8             2=
        9             2-
       10             20
       15            1=0
       20            1-0
     2022         1=11-2
    12345        1-0---0
314159265  1121-1110-1=0
</code></pre>
<p>Based on this process, the SNAFU numbers in the example above can be converted to decimal numbers as follows:</p>
<pre><code> SNAFU  Decimal
1=-0-2     1747
 12111      906
  2=0=      198
    21       11
  2=01      201
   111       31
 20012     1257
   112       32
 1=-1=      353
  1-12      107
    12        7
    1=        3
   122       37
</code></pre>
<p>In decimal, the sum of these numbers is <code>4890</code>.</p>
<p>As you go to input this number on Bob's console, you discover that some buttons you expected are missing. Instead, you are met with buttons labeled <code>=</code>, <code>-</code>, <code>0</code>, <code>1</code>, and <code>2</code>. Bob needs the input value expressed as a SNAFU number, not in decimal.</p>
<p>Reversing the process, you can determine that for the decimal number <code>4890</code>, the SNAFU number you need to supply to Bob's console is <code><em>2=-1=0</em></code>.</p>
<p>The Elves are starting to get cold. <em>What SNAFU number do you supply to Bob's console?</em></p>
</article>


In [35]:
from collections import deque


tests = [
    {
        "name": "Example",
        "s": """
           1=-0-2
            12111
            2=0=
            21
            2=01
            111
            20012
            112
            1=-1=
            1-12
            12
            1=
            122
        """,
        "expected": "2=-1=0",
    },
]


type SNAFU = str


class SNAFU_Utils:
    @staticmethod
    def from_int(digital: int) -> SNAFU:
        if digital == 0:
            return "0"

        snafuvy = "012=-"
        snafu = deque([])
        rest = 0

        while digital > 0:
            digital += rest
            current = digital % 5
            rest = 0
            if current == 3 or current == 4:
                rest = 1
            snafu.appendleft(snafuvy[current])
            digital //= 5

        if rest:
            snafu.appendleft(snafuvy[rest])

        return "".join(snafu)

    @staticmethod
    def to_int(snafu: SNAFU) -> int:
        snafu_digit = {"2": 2, "1": 1, "0": 0, "-": -1, "=": -2}
        result, power = 0, 1

        for ch in snafu:
            result = result * 5
            result += snafu_digit[ch]
        return result

    @staticmethod
    def to_console(s: str) -> SNAFU:
        snafus = [l.strip() for l in s.strip().splitlines()]
        return SNAFU_Utils.from_int(sum(SNAFU_Utils.to_int(sn) for sn in snafus))


digit_to_SNAFU_test = [
    [0, "              0"],
    [1, "              1"],
    [2, "              2"],
    [3, "             1="],
    [4, "             1-"],
    [5, "             10"],
    [6, "             11"],
    [7, "             12"],
    [8, "             2="],
    [9, "             2-"],
    [10, "             20"],
    [15, "            1=0"],
    [20, "            1-0"],
    [2022, "         1=11-2"],
    [12345, "        1-0---0"],
    [314159265, "  1121-1110-1=0"],
]

assert all(
    SNAFU_Utils.from_int(digit) == snafu.strip() for digit, snafu in digit_to_SNAFU_test
)

snafu_digit_test = [
    ["1=-0-2", 1747],
    [" 12111", 906],
    ["  2=0=", 198],
    ["    21", 11],
    ["  2=01", 201],
    ["   111", 31],
    [" 20012", 1257],
    ["   112", 32],
    [" 1=-1=", 353],
    ["  1-12", 107],
    ["    12", 7],
    ["    1=", 3],
    ["   122", 37],
]

assert all(
    SNAFU_Utils.to_int(snafu.strip()) == digit for snafu, digit in snafu_digit_test
)
assert SNAFU_Utils.to_int("20") == 10


@test(tests=tests)
def test_part_I(s: str) -> int:
    return SNAFU_Utils.to_console(s)


Test Example passed, for test_part_I.
Success


In [36]:
with open("../input/day25.txt") as f:
    puzzle = f.read()

print(f"Part I: {SNAFU_Utils.to_console(puzzle)}")

Part I: 2----0=--1122=0=0021


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2----0=--1122=0=0021</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>


<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The <span title="You notice an engraving on the equipment: Balanced Quinary Industries.">hot air balloons</span> quickly carry you to the North Pole. As soon as you land, most of the expedition is escorted directly to a small building attached to the reindeer stables.</p>
<p>The <em>head smoothie chef</em> has just finished warming up the industrial-grade smoothie blender as you arrive. It will take <em class="star">50 stars</em> to fill the blender. The expedition Elves turn their attention to you, and you begin emptying the fruit from your pack onto the table.</p>
<p>As you do, a very young Elf - one you recognize from the expedition team - approaches the table and holds up a single <em class="star">star</em> fruit he found. The head smoothie chef places it in the blender.</p>
<p>Only <em class="star">49 stars</em> to go.</p>
</article>


<link href="style.css" rel="stylesheet"></link>
<main>
<article>
<p>You make a smoothie with all <span class="day-success">fifty stars</span> and deliver it to the reindeer! The sleigh is already warmed up by the time they finish eating.</p>
<p class="aside">Congratulations!  You've finished every puzzle in Advent of Code 2022!  I hope you had as much fun solving them as I had making them for you.  I'd love to hear about your adventure; you can get in touch with me via contact info on <a href="https://was.tl/" target="_blank">my website</a> or through <a href="https://bsky.app/profile/was.tl" target="_blank">Bluesky</a> or <a href="https://hachyderm.io/@ericwastl" target="_blank">Mastodon</a>.</p>
<p class="aside">If you'd like to see more things like this in the future, please consider <a href="/2022/support" target="_blank">supporting</a> Advent of Code and sharing it with others.</p>
<p class="aside">I've <span style="border-bottom:1px dotted #ffff66;" title="Yep, just like that.  There's at least one in the description for each day.">highlighted</span> the easter eggs in each puzzle, just in case you missed any.  Hover your mouse over them, and the easter egg will appear.</p>
<p>You can <span class="share">[Share<span class="share-content">on
  <a href="https://bsky.app/intent/compose?text=I+just+completed+all+25+days+of+Advent+of+Code+2022%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F" target="_blank">Bluesky</a>
  <a href="https://twitter.com/intent/tweet?text=I+just+completed+all+25+days+of+Advent+of+Code+2022%21&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I+just+completed+all+25+days+of+Advent+of+Code+2022%21+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this moment with your friends or <a href="/2022">[Go Check on Your Calendar]</a>.
</p></article>
<style>
.calendar-bkg {
  background: linear-gradient(to bottom, rgba(255,255,255,0) 0%,rgba(255,255,255,.3) 100%);
}
.sf {
  position: fixed;
  animation: anim-sf0 linear 20s infinite;
  z-index: -1;
}
.sf:before {
  content: "*";
}
.sf.sf0 { color: #ffffff; }
.sf.sf1 { color: #cccccc; animation-name: anim-sf1; }
.sf.sf2 { color: #999999; animation-name: anim-sf2; }
.sf.sf3 { color: #666666; animation-name: anim-sf3; }
@keyframes anim-sf0 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(8em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(8em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf1 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(6em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(6em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf2 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(4em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(4em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
@keyframes anim-sf3 {
  0%   { transform: translate(  0,     0) rotate(0)     scale(1.0, 1.0); }
  95%  { transform: translate(2em, 100vh) rotate(1turn) scale(1.0, 1.0); }
  100% { transform: translate(2em, 100vh) rotate(1turn) scale(0.0, 0.0); }
}
</style>
<div class="sf sf2" style="top:-3.88em; left:3.77vw; animation-delay:-7.91s;"></div><div class="sf sf0" style="top:-1.73em; left:24.00vw; animation-delay:-16.59s;"></div><div class="sf sf2" style="top:-3.26em; left:7.38vw; animation-delay:-15.41s;"></div><div class="sf sf2" style="top:-3.30em; left:83.65vw; animation-delay:-7.09s;"></div><div class="sf sf2" style="top:-3.39em; left:38.46vw; animation-delay:-9.08s;"></div><div class="sf sf3" style="top:-4.96em; left:51.22vw; animation-delay:-5.00s;"></div><div class="sf sf3" style="top:-4.32em; left:8.83vw; animation-delay:-13.26s;"></div><div class="sf sf0" style="top:-1.12em; left:37.97vw; animation-delay:-5.62s;"></div><div class="sf sf1" style="top:-2.63em; left:16.63vw; animation-delay:-3.82s;"></div><div class="sf sf0" style="top:-1.23em; left:81.52vw; animation-delay:-18.01s;"></div><div class="sf sf3" style="top:-4.21em; left:24.61vw; animation-delay:-0.39s;"></div><div class="sf sf3" style="top:-4.49em; left:75.43vw; animation-delay:-9.70s;"></div><div class="sf sf0" style="top:-1.78em; left:62.72vw; animation-delay:-10.77s;"></div><div class="sf sf2" style="top:-3.34em; left:67.94vw; animation-delay:-17.91s;"></div><div class="sf sf0" style="top:-1.73em; left:46.53vw; animation-delay:-5.09s;"></div><div class="sf sf0" style="top:-1.29em; left:5.32vw; animation-delay:-10.94s;"></div><div class="sf sf3" style="top:-4.53em; left:43.90vw; animation-delay:-16.55s;"></div><div class="sf sf2" style="top:-3.92em; left:16.82vw; animation-delay:-19.41s;"></div><div class="sf sf3" style="top:-4.96em; left:63.10vw; animation-delay:-4.75s;"></div><div class="sf sf1" style="top:-2.57em; left:51.73vw; animation-delay:-5.09s;"></div><div class="sf sf3" style="top:-4.00em; left:16.04vw; animation-delay:-18.04s;"></div><div class="sf sf0" style="top:-1.28em; left:15.45vw; animation-delay:-12.67s;"></div><div class="sf sf2" style="top:-3.93em; left:15.91vw; animation-delay:-12.05s;"></div><div class="sf sf3" style="top:-4.53em; left:54.69vw; animation-delay:-18.44s;"></div><div class="sf sf0" style="top:-1.85em; left:23.54vw; animation-delay:-2.96s;"></div><div class="sf sf3" style="top:-4.02em; left:63.11vw; animation-delay:-9.23s;"></div><div class="sf sf0" style="top:-1.53em; left:52.05vw; animation-delay:-16.18s;"></div><div class="sf sf2" style="top:-3.27em; left:1.36vw; animation-delay:-14.76s;"></div><div class="sf sf2" style="top:-3.70em; left:74.39vw; animation-delay:-13.73s;"></div><div class="sf sf2" style="top:-3.21em; left:81.46vw; animation-delay:-16.99s;"></div><div class="sf sf2" style="top:-3.80em; left:52.96vw; animation-delay:-6.90s;"></div><div class="sf sf1" style="top:-2.81em; left:42.98vw; animation-delay:-11.27s;"></div><div class="sf sf2" style="top:-3.29em; left:90.29vw; animation-delay:-4.82s;"></div><div class="sf sf0" style="top:-1.90em; left:28.87vw; animation-delay:-15.72s;"></div><div class="sf sf2" style="top:-3.20em; left:9.26vw; animation-delay:-13.92s;"></div><div class="sf sf3" style="top:-4.69em; left:76.24vw; animation-delay:-17.73s;"></div><div class="sf sf0" style="top:-1.75em; left:35.60vw; animation-delay:-8.61s;"></div><div class="sf sf1" style="top:-2.28em; left:67.17vw; animation-delay:-9.40s;"></div><div class="sf sf3" style="top:-4.52em; left:79.53vw; animation-delay:-19.24s;"></div><div class="sf sf2" style="top:-3.25em; left:37.15vw; animation-delay:-17.35s;"></div><div class="sf sf1" style="top:-2.10em; left:72.69vw; animation-delay:-9.79s;"></div><div class="sf sf2" style="top:-3.77em; left:87.94vw; animation-delay:-16.65s;"></div><div class="sf sf3" style="top:-4.40em; left:49.95vw; animation-delay:-4.92s;"></div><div class="sf sf3" style="top:-4.19em; left:83.83vw; animation-delay:-13.35s;"></div><div class="sf sf2" style="top:-3.88em; left:44.80vw; animation-delay:-13.46s;"></div><div class="sf sf2" style="top:-3.73em; left:19.37vw; animation-delay:-5.66s;"></div><div class="sf sf0" style="top:-1.69em; left:25.36vw; animation-delay:-4.01s;"></div><div class="sf sf1" style="top:-2.21em; left:58.45vw; animation-delay:-10.03s;"></div><div class="sf sf0" style="top:-1.37em; left:81.41vw; animation-delay:-16.95s;"></div><div class="sf sf1" style="top:-2.80em; left:78.15vw; animation-delay:-4.86s;"></div><div class="sf sf3" style="top:-4.14em; left:26.23vw; animation-delay:-7.03s;"></div><div class="sf sf1" style="top:-2.76em; left:17.56vw; animation-delay:-2.22s;"></div><div class="sf sf2" style="top:-3.92em; left:60.00vw; animation-delay:-0.48s;"></div><div class="sf sf2" style="top:-3.83em; left:60.02vw; animation-delay:-5.80s;"></div><div class="sf sf0" style="top:-1.13em; left:84.47vw; animation-delay:-1.83s;"></div><div class="sf sf0" style="top:-1.12em; left:33.37vw; animation-delay:-11.85s;"></div><div class="sf sf3" style="top:-4.28em; left:51.42vw; animation-delay:-16.70s;"></div><div class="sf sf3" style="top:-4.06em; left:16.05vw; animation-delay:-19.39s;"></div><div class="sf sf2" style="top:-3.36em; left:79.99vw; animation-delay:-9.09s;"></div><div class="sf sf0" style="top:-1.46em; left:57.68vw; animation-delay:-15.46s;"></div><div class="sf sf0" style="top:-1.04em; left:5.36vw; animation-delay:-4.21s;"></div><div class="sf sf3" style="top:-4.98em; left:30.51vw; animation-delay:-11.96s;"></div><div class="sf sf0" style="top:-1.87em; left:8.75vw; animation-delay:-1.36s;"></div><div class="sf sf2" style="top:-3.55em; left:93.74vw; animation-delay:-11.63s;"></div><div class="sf sf0" style="top:-1.51em; left:90.67vw; animation-delay:-15.14s;"></div><div class="sf sf0" style="top:-1.91em; left:70.19vw; animation-delay:-12.43s;"></div><div class="sf sf2" style="top:-3.66em; left:66.41vw; animation-delay:-17.97s;"></div><div class="sf sf2" style="top:-3.27em; left:90.01vw; animation-delay:-1.99s;"></div><div class="sf sf1" style="top:-2.61em; left:89.51vw; animation-delay:-11.87s;"></div><div class="sf sf1" style="top:-2.46em; left:43.33vw; animation-delay:-6.87s;"></div><div class="sf sf2" style="top:-3.83em; left:50.92vw; animation-delay:-13.55s;"></div><div class="sf sf3" style="top:-4.07em; left:38.63vw; animation-delay:-4.69s;"></div><div class="sf sf1" style="top:-2.64em; left:67.54vw; animation-delay:-12.94s;"></div><div class="sf sf3" style="top:-4.19em; left:40.54vw; animation-delay:-1.88s;"></div><div class="sf sf2" style="top:-3.05em; left:80.79vw; animation-delay:-2.63s;"></div><div class="sf sf2" style="top:-3.04em; left:25.25vw; animation-delay:-4.21s;"></div><div class="sf sf0" style="top:-1.10em; left:6.60vw; animation-delay:-13.23s;"></div><div class="sf sf3" style="top:-4.27em; left:88.54vw; animation-delay:-3.03s;"></div><div class="sf sf1" style="top:-2.71em; left:89.58vw; animation-delay:-7.94s;"></div><div class="sf sf3" style="top:-4.31em; left:95.33vw; animation-delay:-6.26s;"></div><div class="sf sf2" style="top:-3.73em; left:4.75vw; animation-delay:-8.60s;"></div><div class="sf sf1" style="top:-2.86em; left:6.37vw; animation-delay:-19.69s;"></div><div class="sf sf3" style="top:-4.12em; left:46.18vw; animation-delay:-5.05s;"></div><div class="sf sf3" style="top:-4.84em; left:16.14vw; animation-delay:-11.71s;"></div><div class="sf sf1" style="top:-2.57em; left:12.39vw; animation-delay:-19.83s;"></div><div class="sf sf3" style="top:-4.61em; left:72.55vw; animation-delay:-12.65s;"></div><div class="sf sf1" style="top:-2.56em; left:23.71vw; animation-delay:-0.80s;"></div><div class="sf sf0" style="top:-1.85em; left:82.10vw; animation-delay:-8.17s;"></div><div class="sf sf0" style="top:-1.76em; left:92.09vw; animation-delay:-16.21s;"></div><div class="sf sf2" style="top:-3.47em; left:5.30vw; animation-delay:-5.81s;"></div><div class="sf sf1" style="top:-2.11em; left:24.18vw; animation-delay:-8.18s;"></div><div class="sf sf0" style="top:-1.44em; left:50.11vw; animation-delay:-3.33s;"></div><div class="sf sf0" style="top:-1.19em; left:28.07vw; animation-delay:-3.69s;"></div><div class="sf sf1" style="top:-2.54em; left:51.76vw; animation-delay:-8.22s;"></div><div class="sf sf1" style="top:-2.51em; left:59.24vw; animation-delay:-5.29s;"></div><div class="sf sf0" style="top:-1.27em; left:60.11vw; animation-delay:-8.64s;"></div><div class="sf sf1" style="top:-2.41em; left:80.65vw; animation-delay:-7.66s;"></div><div class="sf sf0" style="top:-1.03em; left:25.90vw; animation-delay:-17.95s;"></div><div class="sf sf3" style="top:-4.48em; left:61.61vw; animation-delay:-10.78s;"></div><div class="sf sf2" style="top:-3.17em; left:9.89vw; animation-delay:-7.31s;"></div>
</main>


In [4]:
# print(f"PartI II: {ValleyII(puzzle).go_back_go()}")

<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>895</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
